# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# TO-DO: Create a Keyspace called events
try:
    session.execute("""
                    CREATE KEYSPACE IF NOT EXISTS events
WITH REPLICATION = 
{'class' : 'SimpleStrategy', 'replication_factor' : 1 }
"""
                   )
except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
# TO-DO: Set KEYSPACE to the keyspace specified above 
try:
    session.set_keyspace('events')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### Query 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
  
### Table Design : song_info_by_session
### Primary Key: (sessionId, itemInSession)
### Partition Key: sessionId - Ensures efficient data distribution across nodes by session ID.
### Clustering Column: itemInSession - Enables sorting within each session to quickly retrieve specific items.


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### Table Design :  user_song_session
### Primary Key: ((userId, sessionId), itemInSession)
### Composite Partition Key: (userId, sessionId) - Groups data by user and session for efficient retrieval.
### Clustering Column: itemInSession - Enables sorting by item within each session for easy ordering.
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Table Design : user_by_song
### Partition Key: song - Groups data by song for fast retrieval by song title.
### Clustering Column: userId - Ensures uniqueness within each song and allows retrieval of each user's information.


In [ ]:
# Create table having information about the song based on the session ID

query1 = """CREATE TABLE IF NOT EXISTS song_info_by_session(sessionId INT, itemInSession INT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY (sessionId, itemInSession))"""
session.execute(query1)
query11 = "SELECT length, song, artist FROM song_info_by_session WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query11)
except Exception as e:
    print(e)

for row in rows:
    print(row.length, row.song, row.artist)

query2 = """CREATE TABLE IF NOT EXISTS user_song_session(userId INT, sessionId INT, itemInSession INT, artist TEXT, song TEXT, firstName TEXT, lastName TEXT, PRIMARY KEY ((userId, sessionId), itemInSession, firstName))"""
session.execute(query2)
query21 = "SELECT artist, song, firstName, lastName FROM user_song_session WHERE userId = 10 AND sessionId = 182 ORDER BY itemInSession ASC"
try:
    rows = session.execute(query21)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song)

query3 = """CREATE TABLE IF NOT EXISTS user_by_song(song TEXT, userId INT, firstName TEXT, lastName TEXT, PRIMARY KEY (song, userId))"""
session.execute(query3)
query31 = "SELECT userId, firstName, lastName FROM user_by_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query31)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.userId, row.firstName, row.lastName)
    
# Each query checks for specific conditions based on the expected queries in your project.
# Error Handling: Each query includes basic error handling to catch any issues during execution.
# Print Statements: The print(row) statement will output the retrieved data, allowing you to verify that the data has been inserted correctly.        

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = """INSERT INTO song_info_by_session (sessionId, itemInSession, artist, song, length) VALUES (%s, %s, %s, %s, %s)"""
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        query2 = """INSERT INTO user_song_session (userId, sessionId, itemInSession, artist, song, firstName, lastName) VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        query3 = """INSERT INTO user_by_song (song, userId, firstName, lastName) VALUES (%s, %s, %s, %s)"""
        session.execute(query3, (line[9], int(line[10]), line[1], line[4])) 

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## Verify Query 1: Give me the artist, song title and song's length in the music app history
## for sessionId = 338, and itemInSession = 4
query1 = "SELECT artist, song, length FROM song_info_by_session WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query1)
    for row in rows:
        print(row)
except Exception as e:
    print("Error executing SELECT query:", e)

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
## Verify Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182
query2 = "SELECT artist, song, firstName, lastName FROM user_song_session WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query2)
    for row in rows:
        print(row)
except Exception as e:
    print("Error executing SELECT query:", e)

                    

In [ ]:
## Verify Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "SELECT firstName, lastName FROM user_by_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query3)
    for row in rows:
        print(row)
except Exception as e:
    print("Error executing SELECT query:", e)
                    

### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [ ]:
query1 = "DROP table song_info_by_session"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
query2 = "DROP table user_song_session"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

query3 = "DROP table user_by_song"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()